<a href="https://colab.research.google.com/github/szyluc/Student-Scheduler/blob/main/IT_244_Scheduler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install google-auth
!pip install google-auth-oauthlib
!pip install google-auth-httplib2
!pip install google-api-python-client
!!pip install gradio

In [ ]:
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import os
import gradio as gr

In [ ]:
# Loading FLAN-T5 Model directly from Transformers library, will compare results with GPT-2 Model as well
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

In [ ]:
# For user authentication with Google Calendar
SCOPES = ['https://www.googleapis.com/auth/calendar'] # Requests user for full permission of reading/writing into their Google Calendar

def google_calendar_authentication():
  creds = None
  if os.path.exists('token.json'): # Checking if user session has a token.json
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)

  if not creds or not creds.valid: # If user's credentials don't exist or are invalid, log in
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file("credentials.json", SCOPES)
      creds = flow.run_local_server(port = 0)

    with open("token.json", "w") as token: # Saving user's credentials for future sessions
      token.write(creds.to_json())

  return creds

In [ ]:
# For the textbox interface

def schedule_to_be_generated(user_agenda):
  # Does not do anything yet
  return "Schedule is being generated with the following tasks: {user_agenda}"

interface = gr.Interface(fn = schedule_to_be_generated, inputs = [gr.Textbox(lines = 5, placeholder = "Enter your responsibilities and tasks here", label = "Your Agenda"), gr.Button("Import to Google Calendar")], outputs = "text", title = "Scheduler", description = "Enter your responsibilities and tasks. A personalized schedule will be created for you! Click the 'Import' button to import to your Google Calendar!")

interface.launch()

/usr/local/lib/python3.11/dist-packages/gradio/utils.py:1024: UserWarning: Expected 1 arguments for function <function schedule_to_be_generated at 0x7ef29f88bc40>, received 2.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/gradio/utils.py:1032: UserWarning: Expected maximum 1 arguments for function <function schedule_to_be_generated at 0x7ef29f88bc40>, received 2.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://92916421f247374d80.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
